<a href="https://colab.research.google.com/github/laisab/Inteligencia-Computacional/blob/main/projeto_inteligencia_computacional_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Amadeus-Verbo-FI-Qwen2.5-0.5B-PT-BR-Instruct: https://huggingface.co/amadeusai/Amadeus-Verbo-FI-Qwen2.5-0.5B-PT-BR-Instruct

Portuguese T5 (aka "PTT5"): https://huggingface.co/unicamp-dl/ptt5-base-portuguese-vocab

In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/gpassero/uol-redacoes-xml.git

  Cloning https://github.com/gpassero/uol-redacoes-xml.git to /tmp/pip-req-build-t4_fbgj4
  Running command git clone --filter=blob:none --quiet https://github.com/gpassero/uol-redacoes-xml.git /tmp/pip-req-build-t4_fbgj4
  Resolved https://github.com/gpassero/uol-redacoes-xml.git to commit 94b74fc91c4e7a6b582ebc3708aa0dca2ba12ca6
  Preparing metadata (setup.py) ... done
  Created wheel for uol_redacoes_xml: filename=uol_redacoes_xml-0.2-py3-none-any.whl size=2978835 sha256=f6e4fadcd1802a96cf2d7b95b689229a4b4a243e27ea9119faddcb43bd678e49
  Stored in directory: /tmp/pip-ephem-wheel-cache-l1w4k01s/wheels/c1/e3/ee/70fe667b172b519fa5f401241e6af9b31ab33b05cf715341e5
Successfully built uol_redacoes_xml


In [3]:
import torch, torch.nn.functional as F
import os
import nltk
nltk.download('punkt_tab')
import uol_redacoes_xml
import pandas as pd
from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
essays = uol_redacoes_xml.load()
data = []

for essay in essays:
  data.append({
      'TEMA': essay.prompt.title,
      'REDACAO': essay.text,
      'COMENTARIOS': essay.comments
  })

filename = 'Redacoes.csv'
df_redacoes = pd.DataFrame(data)
df_redacoes.to_csv(filename, index=False)
df_redacoes.drop_duplicates(subset='REDACAO', inplace=True)
df_redacoes.dropna(inplace=True)

In [ ]:
model_id = "amadeusai/AV-FI-Qwen2.5-0.5B-PT-BR-Instruct"

# Tokenização do modelo pré-treinado
tok = AutoTokenizer.from_pretrained(model_id)
# Questão de segurança para modelos criptografados
tok.pad_token = tok.eos_token # LLaMA/Sabiá não tem pad token padrão

# Carregamento do modelo
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True # Permite o uso de modelos com arquiteturas mais recentes que a biblioteca não conhece
)
model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

In [ ]:
lora_cfg = LoraConfig(
    r=8, # Maior capacidade de aprender detalhes complexos devido a um número maior de parâmetros
    lora_alpha=16, # Força do impacto do que o LoRA aprendeu sobre o conhecimento original do modelo (valor alto = novo treino se sobrepõe ao modelo)
    lora_dropout=0.05, # A cada passo do treino, 5% dos neurônios do LoRA são "desligados"
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

# Envolve o modelo base (Sabiá) com o LoRA
model = get_peft_model(model, lora_cfg)
model

In [ ]:
# Template para prompt
IN_FMT = """Corrija a redação do ENEM.\n\nTema: {tema}\n\nRedação:\n{redacao}\n\nFeedback:\n"""

# Conversão do DataFrame para um dataset do tipo Hugging Face
ds = Dataset.from_pandas(df_redacoes)
ds

In [ ]:
# Tamanho máximo da sequência
MAX_LEN = 512

def tok_ex(ex):
  # Formatação da parte de instrução do prompt
  prompt = IN_FMT.format(tema=ex['TEMA'], redacao=ex['REDACAO'])
  # Converte o texto do prompt em tokens sem adição de marcadores de início e fim de frase
  tp = tok(prompt, add_special_tokens=False)
  # Converte o texto da resposta em tokens sem adição de marcadores de início e fim de frase
  tr = tok(ex['COMENTARIOS'] + tok.eos_token, add_special_tokens=False)
  # Concatenar tokens de prompt e de resposta
  ids = (tp['input_ids'] + tr['input_ids'])[:MAX_LEN]
  # Criação da máscara de atenção
  att = [1] * len(ids)
  # Criação dos labels (modelo não deve treinar no prompt, apenas na resposta)
  lab = ([-100]*len(tp["input_ids"]) + tr["input_ids"])[:MAX_LEN] # -100 no prompt: o cálculo de loss dessa parte é ignorado
  # Exemplo tokenizado com os IDs de entrada, máscara de atenção e labels
  return {"input_ids": ids, "attention_mask": att, "labels": lab}

train_ds = ds.map(tok_ex, remove_columns=ds.column_names)
train_ds

In [ ]:
# batch: lista com exemplos do dataset
def collate(batch):
  # Função auxiliar para auxiliar na organização das redações
  def pad_list(lst, pad_val):
    # Maior texto
    maxlen = max(len(x) for x in lst)

    out = []
    # Com os textos menores, adiciona um padding ao final até que fiquem do tamanho do maior
    for x in lst:
      # Conversão para um tensor
      t = torch.tensor(x, dtype=torch.long)
      # Adição do padding
      if t.size(0) < maxlen:
        t = torch.nn.functional.pad(t, (0, maxlen - t.size(0)), value=pad_val)
      out.append(t)

    # Transforma a lista em uma matriz (tensor)
    return torch.stack(out)

  # Extrai os input IDs de todos os exemplos no batch
  ids = [b["input_ids"] for b in batch]
  # Extrai as máscaras de atenção de todos os exemplos no batch
  att = [b["attention_mask"] for b in batch]
  # Extrai os labels de todos os exemplos no batch
  lab = [b["labels"] for b in batch]

  # Texto (input_ids)
  # Preenche o espaço vazio com um token nulo para o modelo saber que aquilo não é uma palavra

  # Atenção (attention_mask)
  # 0 para ignorar, caso contrário, o modelo tentará ler um espaço em branco

  # Gabarito (labels)
  # -100 para ignorar, assim o modelo não é avaliado por acertar um "espaço em branco"
  return{
      "input_ids": pad_list(ids, tok.pad_token_id),
      "attention_mask": pad_list(att, 0),
      "labels": pad_list(lab, -100)
  }

In [ ]:
args = TrainingArguments(
    output_dir="./sabia-lora-redacao",
    num_train_epochs=2, # Número de épocas = quantidade de vezes que o dataset é apresentado para o modelo
    per_device_train_batch_size=2, # Tamanho da lista (batch) por GPU
    gradient_accumulation_steps=4, # Batchs efetivos = 16
    learning_rate=2e-4, # Taxa de aprendizado = velocidade que o modelo aprende
    logging_steps=10, # Frequência com que o treinamento imprime os resultados
    save_steps=50,
    report_to="none", # Desativa o envio de relatórios para ferramentas externas
    fp16=True, # GPU otimizada para trabalhar com 16 bits, acelerando cálculos
    optim="paged_adamw_32bit", # Evita que o treino seja interrompido for falta de recursos (quando a memória chega no limite)
    group_by_length=True # Organiza os batch para que textos de tamanhos parecidos fiquem juntos
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, data_collator=collate)
trainer.model.config

In [ ]:
trainer.train()

In [ ]:
def generate(instr):
  # Formatação da instrução
  prompt = IN_FMT.format(tema=instr['TEMA'], redacao=instr['REDACAO'])
  # Tokenização do prompt e mudança para o serviço do modelo (CPU/GPU)
  x = tok(prompt, return_tensors="pt").to(model.device)

  # Uso do modelo, sem a parte do treinamento
  with torch.no_grad():
    # Apenas a resposta final
    y = model.generate(
        **x, # Desempacota a entrada tokenizada
        max_new_tokens=1024, # Quantidade de tokens a mais que o modelo pode escrever
        do_sample=False, # "Nível de criatividade"
        eos_token_id=tok.eos_token_id, # Parada no token EOS
        pad_token_id=tok.pad_token_id, # Uso do token de padding
        no_repeat_ngram_size=3 # Impede que o modelo repita alguma frase que já foi usada anteriormente
    )

    # Pega os tokens gerados pelo modelo, transforma em palavras e joga fora o que não é usado
    print(tok.decode(y[0], skip_special_tokens=True))

print("\n===EXEMPLO===")
generate(df_redacoes.iloc[22])